In [63]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from random import sample

def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[:2]
    return 'na'

#read the csv into a dataframe
train = pd.read_csv('./data/train_2.csv',sep=',', error_bad_lines=False, warn_bad_lines=True).fillna(0)

train['lang'] = train.Page.map(get_language)
print(Counter(train.lang))

ts_en = train[train.lang=='en']
print("Total Timeseries in Enlighs: %d" % ts_en.shape[0])

ts_en_nonzero = ts_en.loc[(ts_en != 0).all(axis=1), :]
ts_en_nonzero.drop("lang", axis=1, inplace=True)
#dates = [col for col in ts_en.columns if col not in ["Page", "lang"]]
#ts_en["mean"] = ts_en[dates].apply(np.nanmean, axis=1)
print("Total Timeseries with non-zero views in Enlighs: %d" % ts_en_nonzero.shape[0])


Counter({'en': 24108, 'ja': 20431, 'de': 18547, 'na': 17855, 'fr': 17802, 'zh': 17229, 'ru': 15022, 'es': 14069})
Total Timeseries in Enlighs: 24108
Total Timeseries with non-zero views in Enlighs: 16110


/home/amy/Documents/Courses/Fall2017/Model/workspace/lib/python3.5/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [64]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer

http = httplib2.Http()

start_wiki = "National_Football_League"
wiki_prefix = "https://en.wikipedia.org/wiki/"
teams = ['Dallas_Cowboys', 'New_England_Patriots', 'Arizona_Cardinals', 
         'Los_Angeles_Rams', 'Denver_Broncos', 'Pittsburgh_Steelers',
         'Carolina_Panthers', 'Green_Bay_Packers', 'Atlanta_Falcons']


# All agents views 
pages = {p.split('.')[0]:i for i, p in zip(ts_en_nonzero.index, ts_en_nonzero.Page) if p.endswith("_all-agents")}

_, response = http.request(wiki_prefix + start_wiki)
soup = BeautifulSoup(response, "html5lib")
team_pages = set()
for dom in soup.find_all('a'):
    link = dom.get('href') 
    if link is None or not link.startswith("/wiki/"):
        continue
    link = link[len("/wiki/"):]
    if ":" in link or link == "Main_Page":
        continue
    if link in teams and link + "_en" in pages:
        team_pages.add(pages[link + "_en"])
team_pages.add(pages[start_wiki+"_en"])
team_pages = list(team_pages)
print(team_pages)

[72960, 73316, 73254, 74280, 77327, 72943, 73587, 11676, 76796, 75357]


In [65]:
team_ts = ts_en_nonzero.loc[team_pages]
team_ts.to_csv("./data/nfl_teams.csv", header=True, index=False)